## ANNDL Final Project: _Jeopardy!_

In [112]:
import csv
import random
import numpy as np
from sklearn import preprocessing

In [7]:
# Import (value, question, answer) three-ples from CSV.
data = []
with open("/Users/fiordali/Downloads/JEOPARDY_CSV.csv") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        data.append(row[4:])

random.shuffle(data) # Do I have to avoid shuffling the data/recreating the train and test data sets?

In [156]:
# Split data into two randomized groups: testing and training data.
idx = len(data) // 2

train_set = data[:7500]
test_set = data[idx:]

# Remove dollar value and answer from training set.
train_questions = []
for item in train_set:
    train_questions.append(item[1])

In [157]:
# Find length of longest training question by character.
max_len = 0
counter = 0

for question in train_questions:
    for letter in question:
        counter += 1
    if counter > max_len:
        max_len = counter
    counter = 0

## Train LSTM on questions

In [158]:
%matplotlib inline

import sys
import io
import requests as rq
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.optimizers import RMSprop
from collections import Counter

In [159]:
# Q1: What is the purpose of this block? When is `char_indices` used? What about `indices_char`?
chars = sorted(list(set("".join(train_questions))))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

print(char_indices)

{' ': 0, '!': 1, '"': 2, '#': 3, '$': 4, '%': 5, '&': 6, "'": 7, '(': 8, ')': 9, '*': 10, '+': 11, ',': 12, '-': 13, '.': 14, '/': 15, '0': 16, '1': 17, '2': 18, '3': 19, '4': 20, '5': 21, '6': 22, '7': 23, '8': 24, '9': 25, ':': 26, ';': 27, '<': 28, '=': 29, '>': 30, '?': 31, 'A': 32, 'B': 33, 'C': 34, 'D': 35, 'E': 36, 'F': 37, 'G': 38, 'H': 39, 'I': 40, 'J': 41, 'K': 42, 'L': 43, 'M': 44, 'N': 45, 'O': 46, 'P': 47, 'Q': 48, 'R': 49, 'S': 50, 'T': 51, 'U': 52, 'V': 53, 'W': 54, 'X': 55, 'Y': 56, 'Z': 57, '[': 58, ']': 59, '_': 60, 'a': 61, 'b': 62, 'c': 63, 'd': 64, 'e': 65, 'f': 66, 'g': 67, 'h': 68, 'i': 69, 'j': 70, 'k': 71, 'l': 72, 'm': 73, 'n': 74, 'o': 75, 'p': 76, 'q': 77, 'r': 78, 's': 79, 't': 80, 'u': 81, 'v': 82, 'w': 83, 'x': 84, 'y': 85, 'z': 86, '¢': 87, '°': 88, 'é': 89, '–': 90, '’': 91, '“': 92, '”': 93, '…': 94}


In [160]:
# Q2: What is the purpose of this block? What do the `seqlen` and `step` parameters do?
seqlen = max_len # Length in chars of longest question

# Q3: What about this block? What is `x` and what is `y`? Why do they have this dimensionality?
x = np.zeros((len(train_questions), seqlen, len(chars)), dtype=np.bool)
y = np.zeros((len(train_questions), seqlen, len(chars)), dtype=np.bool)
for i, question in enumerate(train_questions):
    # Q3a: What happens in this loop?
    for t, (char_in, char_out) in enumerate(zip(question[:-1], question[1:])):
        x[i, t, char_indices[char_in]] = 1
        y[i, t, char_indices[char_out]] = 1


# Q4: Here we build the model. What does the `return_sequences` argument do? Why the dense layer at the end?
model = Sequential()
model.add(LSTM(128, input_shape=(seqlen, len(chars)), return_sequences=True)) # ret_seq = False because we want abstract feature vector as output
model.add(Dense(len(chars), activation='softmax'))

model.compile(
    loss='categorical_crossentropy',
    optimizer=RMSprop(learning_rate=0.01),
    metrics=['categorical_crossentropy', 'accuracy']
)

In [161]:
model.fit(x, y,
          batch_size=128,
          epochs=50,
#           callbacks=[print_callback],
          verbose=1)

Epoch 1/50
7500/7500 [==============================] - 135s 18ms/step - loss: 0.4979 - categorical_crossentropy: 0.4979 - accuracy: 0.7320
Epoch 2/50
7500/7500 [==============================] - 132s 18ms/step - loss: 0.3869 - categorical_crossentropy: 0.3869 - accuracy: 0.6764
Epoch 3/50
7500/7500 [==============================] - 133s 18ms/step - loss: 0.3424 - categorical_crossentropy: 0.3424 - accuracy: 0.8000
Epoch 4/50
7500/7500 [==============================] - 136s 18ms/step - loss: 0.3204 - categorical_crossentropy: 0.3204 - accuracy: 0.7934
Epoch 5/50
7500/7500 [==============================] - 132s 18ms/step - loss: 0.3052 - categorical_crossentropy: 0.3052 - accuracy: 0.7502
Epoch 6/50
7500/7500 [==============================] - 132s 18ms/step - loss: 0.2932 - categorical_crossentropy: 0.2932 - accuracy: 0.7426
Epoch 7/50
7500/7500 [==============================] - 137s 18ms/step - loss: 0.2836 - categorical_crossentropy: 0.2836 - accuracy: 0.8940
Epoch 8/50
7500/7500

## Train FF on feature vectors from LSTM

In [173]:
# Generate feature vectors for training questions.
# The feature vectors will be the x_train data for the FF network.
for i in range(len(train_questions)):
    print("Evaluating question", i)
    x_pred = np.zeros((1, seqlen, len(chars)))
    for t, char in enumerate(train_questions[i]):
        x_pred[0, t, char_indices[char]] = 1.

    preds = model.predict(x_pred, verbose=0)

Evaluation question 0
(1, 526, 95)
Evaluation question 1
(1, 526, 95)
Evaluation question 2
(1, 526, 95)
Evaluation question 3
(1, 526, 95)
Evaluation question 4
(1, 526, 95)
Evaluation question 5
(1, 526, 95)
Evaluation question 6
(1, 526, 95)
Evaluation question 7
(1, 526, 95)
Evaluation question 8
(1, 526, 95)
Evaluation question 9
(1, 526, 95)
Evaluation question 10
(1, 526, 95)
Evaluation question 11
(1, 526, 95)
Evaluation question 12
(1, 526, 95)
Evaluation question 13
(1, 526, 95)
Evaluation question 14
(1, 526, 95)
Evaluation question 15
(1, 526, 95)
Evaluation question 16
(1, 526, 95)
Evaluation question 17
(1, 526, 95)
Evaluation question 18
(1, 526, 95)
Evaluation question 19
(1, 526, 95)
Evaluation question 20
(1, 526, 95)
Evaluation question 21
(1, 526, 95)
Evaluation question 22
(1, 526, 95)
Evaluation question 23
(1, 526, 95)
Evaluation question 24
(1, 526, 95)
Evaluation question 25
(1, 526, 95)
Evaluation question 26
(1, 526, 95)
Evaluation question 27
(1, 526, 95)
Ev

(1, 526, 95)
Evaluation question 225
(1, 526, 95)
Evaluation question 226
(1, 526, 95)
Evaluation question 227
(1, 526, 95)
Evaluation question 228
(1, 526, 95)
Evaluation question 229
(1, 526, 95)
Evaluation question 230
(1, 526, 95)
Evaluation question 231
(1, 526, 95)
Evaluation question 232
(1, 526, 95)
Evaluation question 233
(1, 526, 95)
Evaluation question 234
(1, 526, 95)
Evaluation question 235
(1, 526, 95)
Evaluation question 236
(1, 526, 95)
Evaluation question 237
(1, 526, 95)
Evaluation question 238
(1, 526, 95)
Evaluation question 239
(1, 526, 95)
Evaluation question 240
(1, 526, 95)
Evaluation question 241
(1, 526, 95)
Evaluation question 242
(1, 526, 95)
Evaluation question 243
(1, 526, 95)
Evaluation question 244
(1, 526, 95)
Evaluation question 245
(1, 526, 95)
Evaluation question 246
(1, 526, 95)
Evaluation question 247
(1, 526, 95)
Evaluation question 248
(1, 526, 95)
Evaluation question 249
(1, 526, 95)
Evaluation question 250
(1, 526, 95)
Evaluation question 251
(

(1, 526, 95)
Evaluation question 449
(1, 526, 95)
Evaluation question 450
(1, 526, 95)
Evaluation question 451
(1, 526, 95)
Evaluation question 452
(1, 526, 95)
Evaluation question 453
(1, 526, 95)
Evaluation question 454
(1, 526, 95)
Evaluation question 455
(1, 526, 95)
Evaluation question 456
(1, 526, 95)
Evaluation question 457
(1, 526, 95)
Evaluation question 458
(1, 526, 95)
Evaluation question 459
(1, 526, 95)
Evaluation question 460
(1, 526, 95)
Evaluation question 461
(1, 526, 95)
Evaluation question 462
(1, 526, 95)
Evaluation question 463
(1, 526, 95)
Evaluation question 464
(1, 526, 95)
Evaluation question 465
(1, 526, 95)
Evaluation question 466
(1, 526, 95)
Evaluation question 467
(1, 526, 95)
Evaluation question 468
(1, 526, 95)
Evaluation question 469
(1, 526, 95)
Evaluation question 470
(1, 526, 95)
Evaluation question 471
(1, 526, 95)
Evaluation question 472
(1, 526, 95)
Evaluation question 473
(1, 526, 95)
Evaluation question 474
(1, 526, 95)
Evaluation question 475
(

(1, 526, 95)
Evaluation question 671
(1, 526, 95)
Evaluation question 672
(1, 526, 95)
Evaluation question 673
(1, 526, 95)
Evaluation question 674
(1, 526, 95)
Evaluation question 675
(1, 526, 95)
Evaluation question 676
(1, 526, 95)
Evaluation question 677
(1, 526, 95)
Evaluation question 678
(1, 526, 95)
Evaluation question 679
(1, 526, 95)
Evaluation question 680
(1, 526, 95)
Evaluation question 681
(1, 526, 95)
Evaluation question 682
(1, 526, 95)
Evaluation question 683
(1, 526, 95)
Evaluation question 684
(1, 526, 95)
Evaluation question 685
(1, 526, 95)
Evaluation question 686
(1, 526, 95)
Evaluation question 687
(1, 526, 95)
Evaluation question 688
(1, 526, 95)
Evaluation question 689
(1, 526, 95)
Evaluation question 690
(1, 526, 95)
Evaluation question 691
(1, 526, 95)
Evaluation question 692
(1, 526, 95)
Evaluation question 693
(1, 526, 95)
Evaluation question 694
(1, 526, 95)
Evaluation question 695
(1, 526, 95)
Evaluation question 696
(1, 526, 95)
Evaluation question 697
(

(1, 526, 95)
Evaluation question 895
(1, 526, 95)
Evaluation question 896
(1, 526, 95)
Evaluation question 897
(1, 526, 95)
Evaluation question 898
(1, 526, 95)
Evaluation question 899
(1, 526, 95)
Evaluation question 900
(1, 526, 95)
Evaluation question 901
(1, 526, 95)
Evaluation question 902
(1, 526, 95)
Evaluation question 903
(1, 526, 95)
Evaluation question 904
(1, 526, 95)
Evaluation question 905
(1, 526, 95)
Evaluation question 906
(1, 526, 95)
Evaluation question 907
(1, 526, 95)
Evaluation question 908
(1, 526, 95)
Evaluation question 909
(1, 526, 95)
Evaluation question 910
(1, 526, 95)
Evaluation question 911
(1, 526, 95)
Evaluation question 912
(1, 526, 95)
Evaluation question 913
(1, 526, 95)
Evaluation question 914
(1, 526, 95)
Evaluation question 915
(1, 526, 95)
Evaluation question 916
(1, 526, 95)
Evaluation question 917
(1, 526, 95)
Evaluation question 918
(1, 526, 95)
Evaluation question 919
(1, 526, 95)
Evaluation question 920
(1, 526, 95)
Evaluation question 921
(

...

(1, 526, 95)
Evaluation question 6541
(1, 526, 95)
Evaluation question 6542
(1, 526, 95)
Evaluation question 6543
(1, 526, 95)
Evaluation question 6544
(1, 526, 95)
Evaluation question 6545
(1, 526, 95)
Evaluation question 6546
(1, 526, 95)
Evaluation question 6547
(1, 526, 95)
Evaluation question 6548
(1, 526, 95)
Evaluation question 6549
(1, 526, 95)
Evaluation question 6550
(1, 526, 95)
Evaluation question 6551
(1, 526, 95)
Evaluation question 6552
(1, 526, 95)
Evaluation question 6553
(1, 526, 95)
Evaluation question 6554
(1, 526, 95)
Evaluation question 6555
(1, 526, 95)
Evaluation question 6556
(1, 526, 95)
Evaluation question 6557
(1, 526, 95)
Evaluation question 6558
(1, 526, 95)
Evaluation question 6559
(1, 526, 95)
Evaluation question 6560
(1, 526, 95)
Evaluation question 6561
(1, 526, 95)
Evaluation question 6562
(1, 526, 95)
Evaluation question 6563
(1, 526, 95)
Evaluation question 6564
(1, 526, 95)
Evaluation question 6565
(1, 526, 95)
Evaluation question 6566
(1, 526, 95)

(1, 526, 95)
Evaluation question 6759
(1, 526, 95)
Evaluation question 6760
(1, 526, 95)
Evaluation question 6761
(1, 526, 95)
Evaluation question 6762
(1, 526, 95)
Evaluation question 6763
(1, 526, 95)
Evaluation question 6764
(1, 526, 95)
Evaluation question 6765
(1, 526, 95)
Evaluation question 6766
(1, 526, 95)
Evaluation question 6767
(1, 526, 95)
Evaluation question 6768
(1, 526, 95)
Evaluation question 6769
(1, 526, 95)
Evaluation question 6770
(1, 526, 95)
Evaluation question 6771
(1, 526, 95)
Evaluation question 6772
(1, 526, 95)
Evaluation question 6773
(1, 526, 95)
Evaluation question 6774
(1, 526, 95)
Evaluation question 6775
(1, 526, 95)
Evaluation question 6776
(1, 526, 95)
Evaluation question 6777
(1, 526, 95)
Evaluation question 6778
(1, 526, 95)
Evaluation question 6779
(1, 526, 95)
Evaluation question 6780
(1, 526, 95)
Evaluation question 6781
(1, 526, 95)
Evaluation question 6782
(1, 526, 95)
Evaluation question 6783
(1, 526, 95)
Evaluation question 6784
(1, 526, 95)

(1, 526, 95)
Evaluation question 6977
(1, 526, 95)
Evaluation question 6978
(1, 526, 95)
Evaluation question 6979
(1, 526, 95)
Evaluation question 6980
(1, 526, 95)
Evaluation question 6981
(1, 526, 95)
Evaluation question 6982
(1, 526, 95)
Evaluation question 6983
(1, 526, 95)
Evaluation question 6984
(1, 526, 95)
Evaluation question 6985
(1, 526, 95)
Evaluation question 6986
(1, 526, 95)
Evaluation question 6987
(1, 526, 95)
Evaluation question 6988
(1, 526, 95)
Evaluation question 6989
(1, 526, 95)
Evaluation question 6990
(1, 526, 95)
Evaluation question 6991
(1, 526, 95)
Evaluation question 6992
(1, 526, 95)
Evaluation question 6993
(1, 526, 95)
Evaluation question 6994
(1, 526, 95)
Evaluation question 6995
(1, 526, 95)
Evaluation question 6996
(1, 526, 95)
Evaluation question 6997
(1, 526, 95)
Evaluation question 6998
(1, 526, 95)
Evaluation question 6999
(1, 526, 95)
Evaluation question 7000
(1, 526, 95)
Evaluation question 7001
(1, 526, 95)
Evaluation question 7002
(1, 526, 95)

(1, 526, 95)
Evaluation question 7194
(1, 526, 95)
Evaluation question 7195
(1, 526, 95)
Evaluation question 7196
(1, 526, 95)
Evaluation question 7197
(1, 526, 95)
Evaluation question 7198
(1, 526, 95)
Evaluation question 7199
(1, 526, 95)
Evaluation question 7200
(1, 526, 95)
Evaluation question 7201
(1, 526, 95)
Evaluation question 7202
(1, 526, 95)
Evaluation question 7203
(1, 526, 95)
Evaluation question 7204
(1, 526, 95)
Evaluation question 7205
(1, 526, 95)
Evaluation question 7206
(1, 526, 95)
Evaluation question 7207
(1, 526, 95)
Evaluation question 7208
(1, 526, 95)
Evaluation question 7209
(1, 526, 95)
Evaluation question 7210
(1, 526, 95)
Evaluation question 7211
(1, 526, 95)
Evaluation question 7212
(1, 526, 95)
Evaluation question 7213
(1, 526, 95)
Evaluation question 7214
(1, 526, 95)
Evaluation question 7215
(1, 526, 95)
Evaluation question 7216
(1, 526, 95)
Evaluation question 7217
(1, 526, 95)
Evaluation question 7218
(1, 526, 95)
Evaluation question 7219
(1, 526, 95)

(1, 526, 95)
Evaluation question 7410
(1, 526, 95)
Evaluation question 7411
(1, 526, 95)
Evaluation question 7412
(1, 526, 95)
Evaluation question 7413
(1, 526, 95)
Evaluation question 7414
(1, 526, 95)
Evaluation question 7415
(1, 526, 95)
Evaluation question 7416
(1, 526, 95)
Evaluation question 7417
(1, 526, 95)
Evaluation question 7418
(1, 526, 95)
Evaluation question 7419
(1, 526, 95)
Evaluation question 7420
(1, 526, 95)
Evaluation question 7421
(1, 526, 95)
Evaluation question 7422
(1, 526, 95)
Evaluation question 7423
(1, 526, 95)
Evaluation question 7424
(1, 526, 95)
Evaluation question 7425
(1, 526, 95)
Evaluation question 7426
(1, 526, 95)
Evaluation question 7427
(1, 526, 95)
Evaluation question 7428
(1, 526, 95)
Evaluation question 7429
(1, 526, 95)
Evaluation question 7430
(1, 526, 95)
Evaluation question 7431
(1, 526, 95)
Evaluation question 7432
(1, 526, 95)
Evaluation question 7433
(1, 526, 95)
Evaluation question 7434
(1, 526, 95)
Evaluation question 7435
(1, 526, 95)

In [142]:
# Many-to-one RNN: variable length question maps to fixed length feature vector
sequence = array(train_questions)
n_in = len(sequence)
sequence = sequence.reshape((1, n_in, 1))

# Define model.
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n_in,1)))
model.add(RepeatVector(n_in))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(1)))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(sequence, sequence, epochs=300, verbose=1)
# demonstrate recreation
yhat = model.predict(sequence, verbose=0)
print(yhat[0,:,0])

Epoch 1/300


ValueError: could not convert string to float: 'Walgreen, Spiegel, Sears'

## References:
* [Understanding LSTMs](https://colah.github.io/posts/2015-08-Understanding-LSTMs/)
* [Emma Boettcher Thesis](https://futurism.com/jeopardy-emma-boettcher-ai-james-holzhauer)
* [A Gentle Introduction to LSTM Autoencoders](https://machinelearningmastery.com/lstm-autoencoders/)
* [LSTM – nuggest for practical application](https://towardsdatascience.com/lstm-nuggets-for-practical-applications-5beef5252092)
* [Understanding Stateful LSTM RNNs Python Keras](https://machinelearningmastery.com/understanding-stateful-lstm-recurrent-neural-networks-python-keras/)
* [Reshape Input Data LSTMs](https://machinelearningmastery.com/reshape-input-data-long-short-term-memory-networks-keras/)
* [How to use return_state](https://www.dlology.com/blog/how-to-use-return_state-or-return_sequences-in-keras/)